#### Here we investigate the relashionship between:
    - mRNA level predictability of a landmark gene 
    and 
    - its known organelle level biological function using GO annotations

In [2]:
import pandas as pd
import numpy as np
def locations_of_substring(string, substring):
    """Return a list of locations of a substring."""
    substring_length = len(substring)    
    def recurse(locations_found, start):
        location = string.find(substring, start)
        if location != -1:
            return recurse(locations_found + [location], location+substring_length)
        else:
            return locations_found
    return recurse([], 0)

In [65]:
# import sys
# sys.path.insert(0, '../utils/') 
# from readProfiles import rename_affyprobe_to_genename
# dfcats2,_=rename_affyprobe_to_genename(dfcats.set_index('ID').T,dfcats.ID.tolist())
# dfcats2.T.reset_index()

gene_cats_bpcc['UP_KW_CELLULAR_COMPONENT'].astype(str).apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('KW')[0] for si in locations_of_substring(x,c)]) if c in x else '').unique()

array(['', 'DNA-directed RNA polymerase,DNA-directed RNA polymerase,'],
      dtype=object)

In [63]:
c='DNA'

'ER'

In [66]:
gene_cats_bpcc['UP_KW_CELLULAR_COMPONENT']

0                                 KW-0496~Mitochondrion,
1                                                    NaN
2      KW-0256~Endoplasmic reticulum,KW-0472~Membrane...
3                                     KW-0963~Cytoplasm,
4        KW-0256~Endoplasmic reticulum,KW-0472~Membrane,
                             ...                        
971                   KW-0539~Nucleus,KW-0963~Cytoplasm,
972                    KW-0472~Membrane,KW-0539~Nucleus,
973                                     KW-0539~Nucleus,
974                                     KW-0539~Nucleus,
975    KW-0137~Centromere,KW-0158~Chromosome,KW-0206~...
Name: UP_KW_CELLULAR_COMPONENT, Length: 976, dtype: object

In [81]:
# gene_cats_bpcc['UP_SEQ_FEATURE']
gene_cats_bpcc['RNA_nucleoli'].unique()

array(['',
       'mrna processing,rna splicing,regulation of mrna export from nucleus,positive regulation of transcription from rna polymerase ii promoter,rna binding,dead/h-box rna helicase binding,',
       'rna binding,nucleolus,',
       'positive regulation of transcription from rna polymerase ii promoter,nucleolus,',
       'maternal placenta development,regulation of mrna stability,positive regulation of transcription from rna polymerase ii promoter,',
       'negative regulation of transcription from rna polymerase ii promoter,regulation of transcription from rna polymerase ii promoter,transcription from rna polymerase ii promoter,positive regulation of transcription from rna polymerase ii promoter,',
       'negative regulation of transcription from rna polymerase ii promoter,regulation of transcription from rna polymerase ii promoter,',
       'regulation of transcription from rna polymerase ii promoter,positive regulation of transcription from rna polymerase ii promoter,rna

In [153]:
# dfcats
# gene_cats_bpcc['UP_SEQ_FEATURE'].str.lower()
# gene_cats_bpcc['GOTERM_BP_DIRECT'].astype(str).str.lower()
# gene_cats_bpcc

# gene_cats_bpcc['UP_KW_CELLULAR_COMPONENT'].unique()
# ci=2
# Channelss_cats=['DNA|Nuclei_AreaShape','RNA','AGP|Cytoplasm_AreaShape|Cells_AreaShape','Mito','ER']
# dfcats.loc[:,dfcats.columns.str.contains(Channelss_cats[ci])]
# dfcats[['max_'+Channelss[ci] for ci in range(len(Channelss))]].idxmax(axis="columns")
dfcats['top_channel']=dfcats[['max_'+Channelss[ci] for ci in range(len(Channelss))]].idxmax(axis="columns")

dfcats.groupby('top_channel').size()

top_channel
max_AGP     238
max_DNA     227
max_ER      182
max_Mito    176
max_RNA     155
dtype: int64

In [300]:
# for c in comps:
for c in ['cytokinesis']:
    print(gene_cats_bpcc['UP_KW_BIOLOGICAL_PROCESS'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('kw')[0] for si in locations_of_substring(x,c)]) if c in x else '').unique())

['' 'steroid metabolic process,'
 'steroid hormone receptor complex assembly,go:0006825~copper ion transport,go:0007566~embryo implantation,go:0010977~negative regulation of neuron projection development,go:0030521~androgen receptor signaling pathway,go:0030850~prostate gland development,go:0031111~negative regulation of microtubule polymerization or depolymerization,go:0031115~negative regulation of microtubule polymerization,go:0031503~protein complex localization,go:0046661~male sex differentiation,go:0061077~chaperone-mediated protein folding,'
 'intracellular steroid hormone receptor signaling pathway,go:0032024~positive regulation of insulin secretion,go:0032962~positive regulation of inositol trisphosphate biosynthetic process,go:0042311~vasodilation,go:0043065~positive regulation of apoptotic process,go:0043280~positive regulation of cysteine-type endopeptidase activity involved in apoptotic process,go:0043401~steroid hormone mediated signaling pathway,go:0043410~positive regul

#### For LUAD dataset:

1 - Read predictability map of categorical features (using MLP model)

2 - Assign the feature categories to compartments/stains

3 - Read functional annotations of the reference set according to DAVIDs output and add columns for each channel
   - Add channel specific annotation to each columns channel
   

In [16]:
import sys
sys.path.insert(0, '../utils/') 
from readProfiles import rename_affyprobe_to_genename
from saveAsNewSheetToExistingFile import saveAsNewSheetToExistingFile
########### 1 ###########
filename='../results/SingleGenePred_cpCategoryMap/cat_scores_maps.xlsx'
saved_scores=pd.read_excel(filename, sheet_name=None)
# which_ds_model='LUAD-9-MLP-ht'
which_ds_model='LUAD-9-MLP-keras-ht'
# which_ds_model='LUAD-9-lasso-ht'
dfcats=saved_scores[which_ds_model].rename(columns={'Unnamed: 0':'ID'})
dfcats=dfcats[dfcats.columns[~dfcats.isna().any()].tolist()]
dfcats2,_=rename_affyprobe_to_genename(dfcats.set_index('ID').T,dfcats.ID.tolist())
dfcats=dfcats2.T.reset_index()

# saved_scores.keys()

########### 2 ###########
Channelss=['DNA','RNA','AGP','Mito','ER']
Channelss_cats=['DNA|Nuclei_AreaShape','RNA','AGP|Cytoplasm_AreaShape|Cells_AreaShape','Mito','ER']
# Channelss_cats=['DNA|Nuclei_AreaShape','RNA','AGP|Cytoplasm_AreaShape','Mito','ER']

for ci in range(len(Channelss)):
    dfcats['max_'+Channelss[ci]]=dfcats.loc[:,dfcats.columns.str.contains(Channelss_cats[ci])].max(axis=1)

    
dfcats['top_channel']=dfcats[['max_'+Channelss[ci] for ci in range(len(Channelss))]].idxmax(axis="columns")
########### 3 ###########
# gene_cats_bpcc=pd.read_csv('./go_BP_CC_MF_DIRECT_921.txt',delimiter='\t')
gene_cats_bpcc=pd.read_csv('./source/GO_bp_cc_mf_direct_LUAD_976.txt',delimiter='\t')
comps=['mitochondri','golgi','membrane','cytoskeleton','actin','endoplasmic','rna','nucleol',\
       'cell division','mitosis','mitotic','cell cycle']#,'cytokine','hormone']
#    UP_KW_BIOLOGICAL_PROCESS bad
# UP_KW_CELLULAR_COMPONENT
# UP_KW_MOLECULAR_FUNCTION #null

# GOTERM_BP_DIRECT
# GOTERM_CC_DIRECT
# GOTERM_MF_DIRECT
for c in comps:
    gene_cats_bpcc[c]=gene_cats_bpcc['GOTERM_CC_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('go')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
    gene_cats_bpcc['GOTERM_MF_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('go')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
    gene_cats_bpcc['GOTERM_BP_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('go')[0] for si in locations_of_substring(x,c)]) if c in x else '')#+\
#     gene_cats_bpcc['UP_KW_BIOLOGICAL_PROCESS'].astype(str).str.lower().apply(lambda x:\
#     ''.join([x[:si].split('~')[-1]+x[si:].split('kw')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
#     gene_cats_bpcc['UP_KW_CELLULAR_COMPONENT'].astype(str).str.lower().apply(lambda x:\
#     ''.join([x[:si].split('~')[-1]+x[si:].split('kw')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
#     gene_cats_bpcc['UP_KW_MOLECULAR_FUNCTION'].astype(str).str.lower().apply(lambda x:\
#     ''.join([x[:si].split('~')[-1]+x[si:].split('kw')[0] for si in locations_of_substring(x,c)]) if c in x else '')#+\
# #     gene_cats_bpcc['UP_SEQ_FEATURE'].astype(str).str.lower().apply(lambda x:\
#     ''.join([x[:si].split(':')[-1]+','+x[si:].split(',')[0] for si in locations_of_substring(x,c)]) if c in x else '')
    
gene_cats_bpcc['RNA_nucleoli']=gene_cats_bpcc['rna']+gene_cats_bpcc['nucleol']
gene_cats_bpcc['DNA']=gene_cats_bpcc['cell division']+gene_cats_bpcc['mitosis']+\
gene_cats_bpcc['mitotic']+gene_cats_bpcc['cell cycle']

gene_cats_bpcc['cytoskeleton-actin']=gene_cats_bpcc['cytoskeleton']+gene_cats_bpcc['actin']+\
gene_cats_bpcc['golgi']+gene_cats_bpcc['membrane']#+gene_cats_bpcc['cytokine']
# gene_cats_bpcc['ER']=gene_cats_bpcc['endoplasmic']
gene_cats_bpcc['mitochondria']=gene_cats_bpcc['mitochondri']#+gene_cats_bpcc['hormone']

gene_cats_bpcc=pd.merge(gene_cats_bpcc,dfcats,how='inner',on='ID')

#########################
Channelss_dict={'DNA':'DNA','RNA_nucleoli':'RNA','cytoskeleton-actin':'AGP','mitochondria':'Mito','endoplasmic':'ER'}
Chan_rev_dict = dict(zip(Channelss_dict.values(),Channelss_dict.keys()))

comps2=['mitochondri','cytoskeleton-actin','endoplasmic','RNA_nucleoli','DNA']
gene_cats_bpcc['any_comps']=''
for co in comps2:
    gene_cats_bpcc['any_comps']=gene_cats_bpcc['any_comps']+gene_cats_bpcc[co]
        
        
        
# gene_cats_bpcc=gene_cats_bpcc[gene_cats_bpcc['any_comps']!=''].reset_index(drop=True)        
from sklearn.metrics import confusion_matrix
from scipy.stats import fisher_exact
# top_bool=(gene_cats_bpcc['top58']==True).values
table2=pd.DataFrame(index=Channelss_dict.keys(),columns=Channelss_dict.values())
table3=pd.DataFrame(index=Channelss,columns=['odds ratio','med_restComp_oddsratio','restComp_oddsratio','anyComp_oddsratio','top-ratio'])
# table3=pd.DataFrame(index=Channelss,columns=['Prevalence','anyComp-Prevalence','noComp-Prevalence','top-ratio'])

table=[]
for c in Channelss:
    print(c)
    c_r=list(set(Channelss)-set([c]))
#     gene_cats_bpcc_highP=gene_cats_bpcc[(gene_cats_bpcc['max_'+c]>0.7)]
#     gene_cats_bpcc_lowP=gene_cats_bpcc[(gene_cats_bpcc['max_'+c]<0.1)]
    
    low_ind=gene_cats_bpcc[['max_'+c]].sort_values(by='max_'+c)[:100].index
    high_ind=gene_cats_bpcc[['max_'+c]].sort_values(by='max_'+c)[-100:].index    
    gene_cats_bpcc_highP=gene_cats_bpcc.loc[high_ind].reset_index(drop=True)
    gene_cats_bpcc_lowP=gene_cats_bpcc.loc[low_ind].reset_index(drop=True)
    
#     gene_cats_bpcc_highP[]
    
#     gene_cats_bpcc2=gene_cats_bpcc[(gene_cats_bpcc['max_'+c]>0.5) | (gene_cats_bpcc['max_'+c]<0)].reset_index(drop=True)
    gene_cats_bpcc2=gene_cats_bpcc.copy()
#     top_bool=(gene_cats_bpcc2['max_'+c]>0.5).values
    
#     gene_cats_bpcc_highP=gene_cats_bpcc[(gene_cats_bpcc['max_'+c]>0.5)]
#     gene_cats_bpcc_lowP=gene_cats_bpcc[(gene_cats_bpcc['max_'+c]<0)]    
    
#     top_bool=(gene_cats_bpcc['max_'+c]>.6).values
    top_bool=(gene_cats_bpcc['max_'+c]<0).values
    
    print(np.sum(gene_cats_bpcc.loc[gene_cats_bpcc['max_'+c]<0,['max_'+Channelss[ci] for ci in range(len(Channelss))]].max(axis=1)>0.3))
#     top_bool=(gene_cats_bpcc['max_'+c]>0.6).values
#     top_bool=((gene_cats_bpcc['top_channel']=='max_'+c) &\
#               (gene_cats_bpcc['max_'+c]>0.4) &\
#              (gene_cats_bpcc[['max_'+cr for cr in c_r]].max(axis=1)<0.4)).values
#     top_bool=((gene_cats_bpcc['max_'+c]-gene_cats_bpcc[['max_'+cr for cr in c_r]].max(axis=1)>0.05) &\
#              (gene_cats_bpcc['max_'+c]>0.2)).values    
    
#     index_of_max=gene_cats_bpcc[(gene_cats_bpcc['top_channel']=='max_'+c)].sort_values(by='max_'+c,ascending=True)\
#     .index[0:45].values
#     top_bool=np.zeros(gene_cats_bpcc.shape[0],).astype(bool)
#     top_bool[index_of_max]=True    
    
#     hhgj
    n_top=sum(top_bool)
    print(n_top)
    
    comps2=['mitochondria','cytoskeleton-actin','endoplasmic','RNA_nucleoli','DNA']
    table1=pd.DataFrame(index=comps2+['any comp','no comp'],columns=['Prevalence','p-value','odds ratio'])
    for co in comps2:
#         ratio_h=gene_cats_bpcc_highP[(gene_cats_bpcc_highP[co]!='')].shape[0]/\
#         gene_cats_bpcc_highP.shape[0]

#         ratio_l=gene_cats_bpcc_lowP[(gene_cats_bpcc_lowP[co]!='')].shape[0]/\
#         gene_cats_bpcc_lowP.shape[0]
# #         fdsfsd
#         print(c,'-',co,': ',ratio_h/ratio_l,gene_cats_bpcc_highP.shape[0],gene_cats_bpcc_lowP.shape[0])
        
        enr_ratio=gene_cats_bpcc2[top_bool & (gene_cats_bpcc2[co]!='')].shape[0]/\
        gene_cats_bpcc2[(gene_cats_bpcc2[co]!='')].shape[0]
        comp_bool=(gene_cats_bpcc2[co]!='').values
        oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, comp_bool))
#         gkkh
    #     print(co, ':',gene_cats_bpcc[(gene_cats_bpcc['top58']==True) & (gene_cats_bpcc[co]!='')].shape[0],\
    #          ', ',gene_cats_bpcc[(gene_cats_bpcc[co]!='')].shape[0])
        
        table1.loc[co,['Prevalence','p-value','odds ratio']]=enr_ratio*100,pvalue,oddsratio
#         print(co,": ",np.round(enr_ratio*100,2),'%     ','pvalue:', np.round(pvalue,2),'  oddsratio:',np.round(oddsratio,2))
        table2.loc[co,c]=oddsratio
         
    
    enr_ratio=gene_cats_bpcc[top_bool & (gene_cats_bpcc['any_comps']!='')].shape[0]/\
    gene_cats_bpcc[(gene_cats_bpcc['any_comps']!='')].shape[0]
    any_oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, (gene_cats_bpcc['any_comps']!='').values))
#     print("any_comps: ",np.round(enr_ratio*100,2),'%     ','pvalue:', np.round(pvalue,2),'  oddsratio:',np.round(oddsratio,2))
    table1.loc['any comp',['Prevalence','p-value','odds ratio']]=enr_ratio*100,pvalue,any_oddsratio
    print(any_oddsratio)

    nocomp_enr_ratio=gene_cats_bpcc[(top_bool) & (gene_cats_bpcc['any_comps']=='')].shape[0]/\
gene_cats_bpcc[(gene_cats_bpcc['any_comps']=='')].shape[0]

    comps2=['mitochondria','cytoskeleton-actin','endoplasmic','RNA_nucleoli','DNA']
    comps2.remove(Chan_rev_dict[c])
    
    med_restComp_oddsratio=table2.loc[comps2,c].median()
    gene_cats_bpcc['rest_comps']=''
    for co in comps2:
        gene_cats_bpcc['rest_comps']=gene_cats_bpcc['rest_comps']+gene_cats_bpcc[co]    
    
    rest_enr_ratio=gene_cats_bpcc[(top_bool) & (gene_cats_bpcc['rest_comps']=='')].shape[0]/\
gene_cats_bpcc[(gene_cats_bpcc['rest_comps']=='')].shape[0]    
    rest_oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, (gene_cats_bpcc['rest_comps']!='').values))
    
    table.append(table1)
#     print('num top ('+ str(n_top)+')/total genes (912): ', np.round((n_top/912)*100,2),'%')
    table3.loc[c,['odds ratio','med_restComp_oddsratio','restComp_oddsratio','anyComp_oddsratio','top-ratio']]=\
    table1.loc[Chan_rev_dict[c],'odds ratio'],med_restComp_oddsratio,rest_oddsratio,\
    any_oddsratio,np.round((n_top/912)*100,2)
    
#     table3.loc[c,['Prevalence','anyComp-Prevalence','noComp-Prevalence','top-ratio']]=table1.loc[Chan_rev_dict[c],'Prevalence'],\
#     enr_ratio*100,nocomp_enr_ratio*100,np.round((n_top/912)*100,2)    
    
#     print(Chan_rev_dict[c],': ',table1.loc[Chan_rev_dict[c],['Prevalence']].values)
    
# # table3['dif']=table3['Prevalence']-table3['anyComp-Prevalence']
table3['dif']=table3['odds ratio']-table3['restComp_oddsratio']
table3['dif2']=table3['odds ratio']-table3['med_restComp_oddsratio']
table3['dif3']=table3['odds ratio']-table3['anyComp_oddsratio']
# print(table3['dif'].min(),table3['dif'].sum())
table3

DNA
0
37
5.224390243902439
RNA
0
52
1.3362175525339925
AGP
0
32
1.3675937122128174
Mito
0
46
1.1570552147239264
ER
1
48
1.5714285714285714


,odds ratio,med_restComp_oddsratio,restComp_oddsratio,anyComp_oddsratio,top-ratio,dif,dif2,dif3
DNA,0.719728,1.289698,6.952986,5.22439,4.06,-6.233258,-0.56997,-4.504663
RNA,1.375026,0.95696,1.164773,1.336218,5.7,0.210253,0.418066,0.038808
AGP,0.74052,0.765185,0.9744,1.367594,3.51,-0.23388,-0.024665,-0.627074
Mito,0.895629,0.965644,0.946092,1.157055,5.04,-0.050463,-0.070015,-0.261426
ER,0.598337,0.987609,1.069565,1.571429,5.26,-0.471229,-0.389272,-0.973092


In [15]:
# a1=(gene_cats_bpcc2[top_bool & (gene_cats_bpcc2[co]!='')].shape[0])/(gene_cats_bpcc2[(top_bool) & (gene_cats_bpcc2[co]=='')].shape[0])
# a2=(gene_cats_bpcc2[(~top_bool) & (gene_cats_bpcc2[co]!='')].shape[0])/(gene_cats_bpcc2[(~top_bool) & (gene_cats_bpcc2[co]=='')].shape[0])

# table3
# gene_cats_bpcc[(gene_cats_bpcc['top_channel']=='max_'+c)].sort_values(by='max_'+c,ascending=False)
# table1
# 
# gene_cats_bpcc[gene_cats_bpcc['max_'+c]<0]
np.sum(gene_cats_bpcc.loc[gene_cats_bpcc['max_'+c]<0,['max_'+Channelss[ci] for ci in range(len(Channelss))]].max(axis=1)>0.3)

1

In [117]:
# high_ind.shape
# gene_cats_bpcc_lowP[(gene_cats_bpcc_lowP[co]!='')]
# table3
# a1/a2
# gene_cats_bpcc[gene_cats_bpcc['any_comps']!=''].reset_index(drop=True)#.shape
x1=table3.astype(float).round(3)[['odds ratio','restComp_oddsratio']]

In [7]:
1e-10

1e-10

In [122]:
# table2.loc[comps2].median()
# pd.concat([table3.astype(float).round(3)[['odds ratio','restComp_oddsratio']],x1],axis=1)

source_data_add='../Figs_Source_Data.xlsx'
if 0:
    saveAsNewSheetToExistingFile(source_data_add,pd.concat([table3.astype(float).round(3)[['odds ratio','restComp_oddsratio']],x1],axis=1),'ExtendedData5')

ExtendedData5  saved!


In [374]:
gene_cats_bpcc2

,ID,Gene Name,Species,GOTERM_BP_DIRECT,GOTERM_CC_DIRECT,GOTERM_MF_DIRECT,KEGG_PATHWAY,UP_KW_BIOLOGICAL_PROCESS,UP_KW_CELLULAR_COMPONENT,UP_KW_MOLECULAR_FUNCTION,...,Cytoplasm_AreaShape,Cells_AreaShape,max_DNA,max_RNA,max_AGP,max_Mito,max_ER,top_channel,any_comps,rest_comps
0,DECR1,"2,4-dienoyl-CoA reductase 1(DECR1)",Homo sapiens,"GO:0006635~fatty acid beta-oxidation,","GO:0005634~nucleus,GO:0005654~nucleoplasm,GO:0...","GO:0008670~2,4-dienoyl-CoA reductase (NADPH) a...",NaN,"KW-0276~Fatty acid metabolism,KW-0443~Lipid me...","KW-0496~Mitochondrion,","KW-0560~Oxidoreductase,",...,0.179192,0.242334,0.259064,0.272621,0.333014,0.302037,0.398891,max_ER,"mitochondrion,mitochondrial matrix,",
1,ADO,2-aminoethanethiol dioxygenase(ADO),Homo sapiens,"GO:0071456~cellular response to hypoxia,","GO:0005829~cytosol,","GO:0005506~iron ion binding,GO:0005515~protein...","hsa00430:Taurine and hypotaurine metabolism,hs...",NaN,NaN,"KW-0223~Dioxygenase,KW-0560~Oxidoreductase,",...,-0.100508,-0.042940,-0.030797,-0.018856,-0.042940,0.010047,-0.004270,max_Mito,"oxidoreductase activity, acting on single dono...","oxidoreductase activity, acting on single dono..."
2,HMGCR,3-hydroxy-3-methylglutaryl-CoA reductase(HMGCR),Homo sapiens,"GO:0006695~cholesterol biosynthetic process,GO...","GO:0005778~peroxisomal membrane,GO:0005783~end...",GO:0004420~hydroxymethylglutaryl-CoA reductase...,"hsa00900:Terpenoid backbone biosynthesis,hsa01...","KW-0152~Cholesterol biosynthesis,KW-0153~Chole...","KW-0256~Endoplasmic reticulum,KW-0472~Membrane...","KW-0560~Oxidoreductase,",...,0.175751,0.176601,0.312376,0.308959,0.258915,0.342495,0.254575,max_Mito,"peroxisomal membrane,endoplasmic reticulum mem...","peroxisomal membrane,endoplasmic reticulum mem..."
3,HMGCS1,3-hydroxy-3-methylglutaryl-CoA synthase 1(HMGCS1),Homo sapiens,"GO:0001889~liver development,GO:0006084~acetyl...","GO:0005737~cytoplasm,GO:0005829~cytosol,",GO:0004421~hydroxymethylglutaryl-CoA synthase ...,"hsa00280:Valine, leucine and isoleucine degrad...","KW-0152~Cholesterol biosynthesis,KW-0153~Chole...","KW-0963~Cytoplasm,","KW-0808~Transferase,",...,0.495392,0.481254,0.566847,0.502503,0.515804,0.567605,0.569166,max_ER,,
4,HACD3,3-hydroxyacyl-CoA dehydratase 3(HACD3),Homo sapiens,"GO:0006633~fatty acid biosynthetic process,GO:...","GO:0005783~endoplasmic reticulum,GO:0005789~en...","GO:0005096~GTPase activator activity,GO:000551...","hsa00062:Fatty acid elongation,hsa01040:Biosyn...","KW-0275~Fatty acid biosynthesis,KW-0276~Fatty ...","KW-0256~Endoplasmic reticulum,KW-0472~Membrane,","KW-0456~Lyase,",...,0.040313,-0.051127,0.063998,0.070532,0.040313,0.086491,0.179419,max_ER,"endoplasmic reticulum membrane,integral compon...","endoplasmic reticulum membrane,integral compon..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,ZNF395,zinc finger protein 395(ZNF395),Homo sapiens,"GO:0006355~regulation of transcription, DNA-te...","GO:0005634~nucleus,GO:0005654~nucleoplasm,GO:0...",GO:0000978~RNA polymerase II core promoter pro...,NaN,"KW-0804~Transcription,KW-0805~Transcription re...","KW-0539~Nucleus,KW-0963~Cytoplasm,","KW-0238~DNA-binding,",...,0.104552,0.152037,0.189749,0.151408,0.152037,0.112444,0.140626,max_DNA,rna polymerase ii core promoter proximal regio...,rna polymerase ii core promoter proximal regio...
972,ZNF451,zinc finger protein 451(ZNF451),Homo sapiens,"GO:0010468~regulation of gene expression,GO:00...","GO:0005634~nucleus,GO:0016021~integral compone...","GO:0003714~transcription corepressor activity,...",NaN,"KW-0804~Transcription,KW-0805~Transcription re...","KW-0472~Membrane,KW-0539~Nucleus,","KW-0808~Transferase,",...,0.411780,0.336247,0.522736,0.523008,0.485401,0.567924,0.462915,max_Mito,"integral component of membrane,positive regula...","integral component of membrane,positive regula..."
973,ZNF586,zinc finger protein 586(ZNF586),Homo sapiens,"GO:0006355~regulation of transcription, DNA-te...","GO:0005634~nucleus,",GO:0000978~RNA polymer

In [377]:
# high_ind=gene_cats_bpcc[['max_'+c]].sort_values(by='max_'+c)[:-100].index
# low_ind=gene_cats_bpcc[['max_'+c]].sort_values(by='max_'+c)[-100:].index
comps2

['mitochondria', 'cytoskeleton-actin', 'RNA_nucleoli', 'DNA']

In [380]:
# gene_cats_bpcc.loc[high_ind]
comps2=['mitochondria','cytoskeleton-actin','endoplasmic','RNA_nucleoli','DNA']
for co in comps2:
#         ratio_h=gene_cats_bpcc_highP[(gene_cats_bpcc_highP[co]!='')].shape[0]/\
#         gene_cats_bpcc_highP.shape[0]

#         ratio_l=gene_cats_bpcc_lowP[(gene_cats_bpcc_lowP[co]!='')].shape[0]/\
#         gene_cats_bpcc_lowP.shape[0]
# #         fdsfsd
#         print(c,'-',co,': ',ratio_h/ratio_l,gene_cats_bpcc_highP.shape[0],gene_cats_bpcc_lowP.shape[0])

    print(co,gene_cats_bpcc2[(gene_cats_bpcc2[co]!='')].shape[0])


mitochondria 185
cytoskeleton-actin 617
endoplasmic 156
RNA_nucleoli 361
DNA 205


In [240]:
# Channelss.index(c)
# gene_cats_bpcc2[(gene_cats_bpcc2[co]!='')].shape[0]
gene_cats_bpcc[['max_'+c]].sort_values(by='max_'+c)[:100].index

Int64Index([476, 241,  55, 908, 436, 149, 699, 435, 125, 299, 744, 273,  69,
            607, 303, 605, 765, 974, 459,  92, 183, 836, 180, 352, 143, 590,
            130, 424, 655, 775, 637, 156, 227, 832, 871,  26, 573,  90, 622,
            575,  17,   1, 843, 550, 814, 326, 890,  45, 900, 677, 135, 569,
            583, 909, 855, 828, 454, 280, 327, 894, 662, 973, 615, 282, 357,
            404,  63, 120, 912, 597,  28, 861, 721, 649, 617, 751,  64, 591,
            845, 586, 342, 155, 937, 428, 929, 713,  70, 959, 127, 527, 340,
            341, 173, 228,  99, 383, 300, 132, 659, 792],
           dtype='int64')

In [241]:
gene_cats_bpcc[['max_'+c]].sort_values(by='max_'+c)[-100:].index

Int64Index([166, 472, 201,  50, 759, 904, 377, 687, 916, 652, 886, 621, 581,
            728, 482, 626,  44, 896, 643, 789, 535, 588, 623, 932, 244, 154,
            359, 369, 388, 703, 504, 666, 313, 786,  78, 641,  83, 927, 253,
            256, 833, 372,  68, 421, 815, 532, 878, 115, 754, 438, 735,  56,
            498, 221, 324, 730, 701, 598, 829, 555, 821, 891, 661, 401, 934,
            289, 457, 618, 456, 370,  81, 243, 907, 955, 624, 323, 692, 638,
            542, 443,  88, 411, 295, 817, 400, 145, 523, 260, 366, 540, 838,
            285, 642, 939, 835, 848, 328, 948,  79, 426],
           dtype='int64')

In [356]:
# top_bool
# (gene_cats_bpcc['top_channel']=='max_'+c) & (gene_cats_bpcc['max_'+c]>0.6)
# top_bool.shape
# index_of_max
# list(set(Channelss)-set([c]))
# gene_cats_bpcc2=gene_cats_bpcc[(gene_cats_bpcc['max_'+c]>0.6) | (gene_cats_bpcc['max_'+c]<0.1)].reset_index(drop=True)


In [170]:
index_of_max=gene_cats_bpcc[(gene_cats_bpcc['top_channel']=='max_'+c)].sort_values(by='max_'+c,ascending=False).index[0:100].values
top_bool=np.zeros(976,).astype(bool)
top_bool[index_of_max]=True

In [177]:
# top_bool=np.zeros(976,).astype(bool)
# top_bool[index_of_max]=True
top_bool[427]

False

In [90]:
# gene_cats_bpcc.shape

luad_genes_to_inspect_ims=['MLLT11','FAM20B','SQSTM1','TXNRD1']

(976, 52)

In [95]:
gene_cats_bpcc[gene_cats_bpcc['GOTERM_MF_DIRECT'].isnull()].shape

(17, 52)

In [45]:
pd.read_csv('./source/GO_bp_cc_mf_direct_LUAD_975.txt',delimiter='\t').shape

(975, 6)

In [85]:
table3

,odds ratio,restComp_oddsratio,anyComp_oddsratio,top-ratio,dif
DNA,2.187816,0.64314,1.025543,9.1,1.544676
RNA,0.952168,1.004431,0.993456,9.76,-0.052264
AGP,0.851337,1.041846,0.992218,10.64,-0.190509
Mito,0.577881,1.381192,1.271478,9.76,-0.803311
ER,1.040071,0.828983,0.83908,9.32,0.211089


In [51]:
table3

,odds ratio,restComp_oddsratio,anyComp_oddsratio,top-ratio,dif
DNA,2.167224,0.855631,1.196811,11.51,1.311593
RNA,1.088565,1.041325,1.041498,12.17,0.04724
AGP,0.769428,1.127649,1.035857,13.05,-0.358221
Mito,0.687255,1.219174,1.196811,11.51,-0.531919
ER,1.08498,0.961039,0.9819,11.62,0.123941


In [8]:
table3

,odds ratio,restComp_oddsratio,anyComp_oddsratio,top-ratio,dif
DNA,0.873695,1.629147,1.724173,18.2,-0.755452
RNA,1.197029,1.262254,1.609984,19.96,-0.065225
AGP,0.967497,1.16015,1.818722,16.01,-0.192653
Mito,0.979993,1.309076,1.444275,18.42,-0.329083
ER,0.831579,1.236551,1.541405,20.61,-0.404972


In [11]:
table3

,odds ratio,restComp_oddsratio,anyComp_oddsratio,top-ratio,dif
DNA,0.793062,1.353643,1.419434,20.5,-0.560581
RNA,1.291008,1.317432,1.339849,20.83,-0.026424
AGP,0.834736,1.322061,1.637884,18.75,-0.487325
Mito,1.174962,1.359223,1.566091,20.72,-0.184261
ER,1.050249,1.112284,1.320451,22.92,-0.062034


In [359]:
print(table3.to_latex())

\begin{tabular}{llllll}
\toprule
{} & odds ratio & restComp\_oddsratio & anyComp\_oddsratio & top-ratio &       dif \\
\midrule
DNA  &   0.836969 &           1.218462 &          1.126352 &     19.96 & -0.381493 \\
RNA  &   1.127582 &           1.136146 &          1.217852 &     21.93 & -0.008563 \\
AGP  &   1.053636 &           1.057746 &          1.270029 &     17.32 &  -0.00411 \\
Mito &   1.005369 &           1.282287 &          1.289183 &     20.18 & -0.276917 \\
ER   &   0.890449 &           1.049286 &          1.177658 &     22.26 & -0.158838 \\
\bottomrule
\end{tabular}



## GO terms search for overlap of highly predictable genes (top 58)

In [2]:
# top58=pd.read_csv('./top_58_common.txt',header=None)[0].tolist()
# gene_cats_bpcc=pd.read_csv('./go_bp_cc_D2021_each_gene_cat.txt',delimiter='\t')
import sys
sys.path.insert(0, '../utils/') 
from readProfiles import rename_affyprobe_to_genename
from saveAsNewSheetToExistingFile import saveAsNewSheetToExistingFile
top58=pd.read_csv('./source/top_59_atleast_topIn3.txt',header=None)[0].tolist()
gene_cats_bpcc=pd.read_csv('./source/GO_bp_cc_mf_direct_intersection_782.txt',delimiter='\t')

# comps=['mitochondri','Golgi','membrane','cytoskeleton','actin','endoplasmic','RNA','nucleol','cell division','mitosis','mitotic','cell cycle']

# for c in comps:
#     gene_cats_bpcc[c]=gene_cats_bpcc['GOTERM_BP_DIRECT'].astype(str).apply(lambda x: x if c in x else '')+\
#     gene_cats_bpcc['GOTERM_CC_DIRECT'].astype(str).apply(lambda x: x[:x.find(c)].split('~')[-1]+x[x.find(c):].split('GO')[0] if c in x else '')
comps=['mitochondri','golgi','membrane','cytoskeleton','actin','endoplasmic','rna','nucleol',\
       'cell division','mitosis','mitotic','cell cycle','cytokine','hormone']

for c in comps:
    gene_cats_bpcc[c]=gene_cats_bpcc['GOTERM_BP_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('GO')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
    gene_cats_bpcc['GOTERM_CC_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('GO')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
    gene_cats_bpcc['GOTERM_MF_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('GO')[0] for si in locations_of_substring(x,c)]) if c in x else '')
    
gene_cats_bpcc['RNA_nucleoli']=gene_cats_bpcc['rna']+gene_cats_bpcc['nucleol']
gene_cats_bpcc['DNA']=gene_cats_bpcc['cell division']+gene_cats_bpcc['mitosis']+gene_cats_bpcc['mitotic']+gene_cats_bpcc['cell cycle']

gene_cats_bpcc['cytoskeleton-actin']=gene_cats_bpcc['cytoskeleton']+gene_cats_bpcc['actin']

# gene_cats_bpcc['RNA_nucleoli']=gene_cats_bpcc[' rna']+gene_cats_bpcc['nucleol']
# gene_cats_bpcc['DNA']=gene_cats_bpcc['cell division']+gene_cats_bpcc['mitosis']+gene_cats_bpcc['mitotic']+gene_cats_bpcc['cell cycle']

# gene_cats_bpcc['cytoskeleton-actin']=gene_cats_bpcc['cytoskeleton']+gene_cats_bpcc['actin']+\
# gene_cats_bpcc['golgi']+gene_cats_bpcc['membrane']+gene_cats_bpcc['cytokine']
# gene_cats_bpcc['ER']=gene_cats_bpcc['endoplasmic']
# gene_cats_bpcc['mitochondria']=gene_cats_bpcc['mitochondri']#+gene_cats_bpcc['hormone']



# gene_cats_bpcc['RNA_nucleoli']=gene_cats_bpcc['GOTERM_BP_DIRECT'].astype(str).apply(lambda x: x if 'RNA' in x or 'nucleoli' in x else '')+\
# gene_cats_bpcc['GOTERM_CC_DIRECT'].astype(str).apply(lambda x: x if 'RNA' in x or 'nucleoli' in x else '')

# gene_cats_bpcc['DNA']=gene_cats_bpcc['GOTERM_BP_DIRECT'].astype(str).apply(lambda x: x if 'cell division' in x or 'mitosis' in x or 'mitotic' in x or 'cell cycle' in x else '')+\
# gene_cats_bpcc['GOTERM_CC_DIRECT'].astype(str).apply(lambda x: x if 'cell division' in x or 'mitosis' in x or 'mitotic' in x or 'cell cycle' in x else '')

# gene_cats_bpcc['DNA']=gene_cats_bpcc[c]=gene_cats_bpcc['GOTERM_BP_DIRECT'].astype(str).apply(lambda x: x if 'cell division' in x or 'mitosis' in x or 'cell cycle' in x else '')+\
# gene_cats_bpcc['GOTERM_CC_DIRECT'].astype(str).apply(lambda x: x if 'cell division' in x or 'mitosis' in x or 'cell cycle' in x else '')


gene_cats_bpcc.loc[gene_cats_bpcc['ID'].isin(top58),'top58']=True



#####################################
comps2=['mitochondri','golgi','membrane','cytoskeleton-actin','endoplasmic','RNA_nucleoli','DNA']
from sklearn.metrics import confusion_matrix
from scipy.stats import fisher_exact
top_bool=(gene_cats_bpcc['top58']==True).values

table1=pd.DataFrame(index=comps2+['any comp','no comp'],columns=['Prevalence','p-value','odds ratio'])
for co in comps2:
    enr_ratio=gene_cats_bpcc[(gene_cats_bpcc['top58']==True) & (gene_cats_bpcc[co]!='')].shape[0]/\
    gene_cats_bpcc[(gene_cats_bpcc[co]!='')].shape[0]
    comp_bool=(gene_cats_bpcc[co]!='').values
    oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, comp_bool))
#     print(co, ':',gene_cats_bpcc[(gene_cats_bpcc['top58']==True) & (gene_cats_bpcc[co]!='')].shape[0],\
#          ', ',gene_cats_bpcc[(gene_cats_bpcc[co]!='')].shape[0])

    table1.loc[co,['Prevalence','p-value','odds ratio']]=enr_ratio*100,pvalue,oddsratio
    print(co,": ",np.round(enr_ratio*100,2),'%     ','pvalue:', np.round(pvalue,2),'  oddsratio:',np.round(oddsratio,2))
    
gene_cats_bpcc['any_comps']=''
for co in comps2:
    gene_cats_bpcc['any_comps']=gene_cats_bpcc['any_comps']+gene_cats_bpcc[co]
     
    
enr_ratio=gene_cats_bpcc[(gene_cats_bpcc['top58']==True) & (gene_cats_bpcc['any_comps']!='')].shape[0]/\
gene_cats_bpcc[(gene_cats_bpcc['any_comps']!='')].shape[0]
oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, (gene_cats_bpcc['any_comps']!='').values))
print("any_comps: ",np.round(enr_ratio*100,2),'%     ','pvalue:', np.round(pvalue,2),'  oddsratio:',np.round(oddsratio,2))
table1.loc['any comp',['Prevalence','p-value','odds ratio']]=enr_ratio*100,pvalue,oddsratio


enr_ratio=gene_cats_bpcc[(gene_cats_bpcc['top58']==True) & (gene_cats_bpcc['any_comps']=='')].shape[0]/\
gene_cats_bpcc[(gene_cats_bpcc['any_comps']=='')].shape[0]
oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, (gene_cats_bpcc['any_comps']=='').values))
print("no comps: ",np.round(enr_ratio*100,2),'%     ','pvalue:', np.round(pvalue,2),'  oddsratio:',np.round(oddsratio,2))
table1.loc['no comp',['Prevalence','p-value','odds ratio']]=enr_ratio*100,pvalue,oddsratio

# print('num top (52)/total genes (782): ', np.round((52/782)*100,2),'%')

source_data_add='../Figs_Source_Data.xlsx'
if 0:
    saveAsNewSheetToExistingFile(source_data_add,table1,'ExtendedData6')

# gene_cats_bpcc.to_csv('./GO/go_bp_cc_D2021_each_gene_cat_completed.csv',index=False)
# gene_cats_bpcc.to_csv('./source/GO_bp_cc_mf_direct_intersection_782_completed.csv',index=False)


mitochondri :  9.21 %      pvalue: 0.2   oddsratio: 1.58
golgi :  4.9 %      pvalue: 0.53   oddsratio: 0.69
membrane :  7.17 %      pvalue: 0.56   oddsratio: 1.22
cytoskeleton-actin :  9.52 %      pvalue: 0.21   oddsratio: 1.59
endoplasmic :  6.78 %      pvalue: 1.0   oddsratio: 1.02
RNA_nucleoli :  4.96 %      pvalue: 0.22   oddsratio: 0.64
DNA :  10.77 %      pvalue: 0.05   oddsratio: 1.95
any_comps:  7.18 %      pvalue: 0.13   oddsratio: 2.5
no comps:  3.0 %      pvalue: 0.13   oddsratio: 0.4


In [5]:
table1['odds ratio'].astype(float).round(2)

mitochondri           1.58
golgi                 0.69
membrane              1.22
cytoskeleton-actin    1.59
endoplasmic           1.02
RNA_nucleoli          0.64
DNA                   1.95
any comp              2.50
no comp               0.40
Name: odds ratio, dtype: float64

In [12]:
table1['odds ratio'].astype(float).round(2).values

array([1.58, 0.69, 1.22, 1.59, 1.02, 0.64, 1.95, 2.5 , 0.4 ])

In [29]:
gene_cats_bpcc[gene_cats_bpcc['ID'].isin(top58)].shape

(52, 23)

In [31]:
# gene_cats_bpcc.shape
mitochondri :  9.21 %      pvalue: 0.2   oddsratio: 1.58
golgi :  4.9 %      pvalue: 0.53   oddsratio: 0.69
membrane :  7.17 %      pvalue: 0.56   oddsratio: 1.22
cytoskeleton-actin :  9.52 %      pvalue: 0.21   oddsratio: 1.59
endoplasmic :  6.78 %      pvalue: 1.0   oddsratio: 1.02
RNA_nucleoli :  5.68 %      pvalue: 0.73   oddsratio: 0.81
DNA :  10.77 %      pvalue: 0.05   oddsratio: 1.95
any_comps:  7.27 %      pvalue: 0.12   oddsratio: 2.31
no comps:  3.28 %      pvalue: 0.12   oddsratio: 0.43
num top (52)/total genes (782):  6.65 %

(782, 23)

In [ ]:
import sys
sys.path.insert(0, '../utils/') 
from readProfiles import rename_affyprobe_to_genename
########### 1 ###########
filename='../results/SingleGenePred_cpCategoryMap/cat_scores_maps.xlsx'
saved_scores=pd.read_excel(filename, sheet_name=None)
# which_ds_model='LUAD-9-MLP-ht'
which_ds_model='LUAD-9-MLP-keras-ht'
# which_ds_model='LUAD-9-lasso-ht'
dfcats=saved_scores[which_ds_model].rename(columns={'Unnamed: 0':'ID'})
dfcats=dfcats[dfcats.columns[~dfcats.isna().any()].tolist()]
dfcats2,_=rename_affyprobe_to_genename(dfcats.set_index('ID').T,dfcats.ID.tolist())
dfcats=dfcats2.T.reset_index()

# saved_scores.keys()

########### 2 ###########
Channelss=['DNA','RNA','AGP','Mito','ER']
Channelss_cats=['DNA|Nuclei_AreaShape','RNA','AGP|Cytoplasm_AreaShape|Cells_AreaShape','Mito','ER']
# Channelss_cats=['DNA|Nuclei_AreaShape','RNA','AGP|Cytoplasm_AreaShape','Mito','ER']

for ci in range(len(Channelss)):
    dfcats['max_'+Channelss[ci]]=dfcats.loc[:,dfcats.columns.str.contains(Channelss_cats[ci])].max(axis=1)

    
dfcats['top_channel']=dfcats[['max_'+Channelss[ci] for ci in range(len(Channelss))]].idxmax(axis="columns")
########### 3 ###########
# gene_cats_bpcc=pd.read_csv('./go_BP_CC_MF_DIRECT_921.txt',delimiter='\t')
gene_cats_bpcc=pd.read_csv('./source/GO_bp_cc_mf_direct_LUAD_976.txt',delimiter='\t')
comps=['mitochondri','golgi','membrane','cytoskeleton','actin','endoplasmic',' rna','nucleol',\
       'cell division','mitosis','mitotic','cell cycle','cytokine','hormone']
#    UP_KW_BIOLOGICAL_PROCESS bad
# UP_KW_CELLULAR_COMPONENT
# UP_KW_MOLECULAR_FUNCTION #null

# GOTERM_BP_DIRECT
# GOTERM_CC_DIRECT
# GOTERM_MF_DIRECT
for c in comps:
    gene_cats_bpcc[c]=gene_cats_bpcc['GOTERM_CC_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('go')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
    gene_cats_bpcc['GOTERM_MF_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('go')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
    gene_cats_bpcc['GOTERM_BP_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('go')[0] for si in locations_of_substring(x,c)]) if c in x else '')#+\
#     gene_cats_bpcc['UP_KW_BIOLOGICAL_PROCESS'].astype(str).str.lower().apply(lambda x:\
#     ''.join([x[:si].split('~')[-1]+x[si:].split('kw')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
#     gene_cats_bpcc['UP_KW_CELLULAR_COMPONENT'].astype(str).str.lower().apply(lambda x:\
#     ''.join([x[:si].split('~')[-1]+x[si:].split('kw')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
#     gene_cats_bpcc['UP_KW_MOLECULAR_FUNCTION'].astype(str).str.lower().apply(lambda x:\
#     ''.join([x[:si].split('~')[-1]+x[si:].split('kw')[0] for si in locations_of_substring(x,c)]) if c in x else '')#+\
#     gene_cats_bpcc['UP_SEQ_FEATURE'].astype(str).str.lower().apply(lambda x:\
#     ''.join([x[:si].split(':')[-1]+','+x[si:].split(',')[0] for si in locations_of_substring(x,c)]) if c in x else '')
    
gene_cats_bpcc['RNA_nucleoli']=gene_cats_bpcc[' rna']+gene_cats_bpcc['nucleol']
gene_cats_bpcc['DNA']=gene_cats_bpcc['cell division']+gene_cats_bpcc['mitosis']+gene_cats_bpcc['mitotic']+gene_cats_bpcc['cell cycle']

gene_cats_bpcc['cytoskeleton-actin']=gene_cats_bpcc['cytoskeleton']+gene_cats_bpcc['actin']+\
gene_cats_bpcc['golgi']+gene_cats_bpcc['membrane']+gene_cats_bpcc['cytokine']
# gene_cats_bpcc['ER']=gene_cats_bpcc['endoplasmic']
gene_cats_bpcc['mitochondria']=gene_cats_bpcc['mitochondri']+gene_cats_bpcc['hormone']

gene_cats_bpcc=pd.merge(gene_cats_bpcc,dfcats,how='inner',on='ID')

#########################
Channelss_dict={'DNA':'DNA','RNA_nucleoli':'RNA','cytoskeleton-actin':'AGP','mitochondria':'Mito','endoplasmic':'ER'}
Chan_rev_dict = dict(zip(Channelss_dict.values(),Channelss_dict.keys()))

comps2=['mitochondri','cytoskeleton-actin','endoplasmic','RNA_nucleoli','DNA']
gene_cats_bpcc['any_comps']=''
for co in comps2:
    gene_cats_bpcc['any_comps']=gene_cats_bpcc['any_comps']+gene_cats_bpcc[co]
        
from sklearn.metrics import confusion_matrix
from scipy.stats import fisher_exact
# top_bool=(gene_cats_bpcc['top58']==True).values
table2=pd.DataFrame(index=Channelss_dict.keys(),columns=Channelss_dict.values())
table3=pd.DataFrame(index=Channelss,columns=['odds ratio','restComp_oddsratio','anyComp_oddsratio','top-ratio'])
# table3=pd.DataFrame(index=Channelss,columns=['Prevalence','anyComp-Prevalence','noComp-Prevalence','top-ratio'])

table=[]
for c in Channelss:
    print(c)
    c_r=list(set(Channelss)-set([c]))
#     gene_cats_bpcc_highP=gene_cats_bpcc[(gene_cats_bpcc['max_'+c]>0.7)]
#     gene_cats_bpcc_lowP=gene_cats_bpcc[(gene_cats_bpcc['max_'+c]<0.1)]
    
#     low_ind=gene_cats_bpcc[['max_'+c]].sort_values(by='max_'+c)[:100].index
#     high_ind=gene_cats_bpcc[['max_'+c]].sort_values(by='max_'+c)[-100:].index    
#     gene_cats_bpcc_highP=gene_cats_bpcc.loc[high_ind].reset_index(drop=True)
#     gene_cats_bpcc_lowP=gene_cats_bpcc.loc[low_ind].reset_index(drop=True)
    
#     gene_cats_bpcc_highP[]
    
#     gene_cats_bpcc2=gene_cats_bpcc[(gene_cats_bpcc['max_'+c]>0.6) | (gene_cats_bpcc['max_'+c]<0.1)].reset_index(drop=True)
    gene_cats_bpcc2=gene_cats_bpcc.copy()
#     top_bool=(gene_cats_bpcc2['max_'+c]>0.6).values
#     top_bool=(gene_cats_bpcc['max_'+c]<0.1).values
#     top_bool=(gene_cats_bpcc['max_'+c]>0.6).values
#     top_bool=((gene_cats_bpcc['top_channel']=='max_'+c) &\
#               (gene_cats_bpcc['max_'+c]>0.4) &\
#              (gene_cats_bpcc[['max_'+cr for cr in c_r]].max(axis=1)<0.4)).values
    top_bool=((gene_cats_bpcc['max_'+c]-gene_cats_bpcc[['max_'+cr for cr in c_r]].max(axis=1)>0.05) &\
             (gene_cats_bpcc['max_'+c]>0.3)).values    
    
#     index_of_max=gene_cats_bpcc[(gene_cats_bpcc['top_channel']=='max_'+c)].sort_values(by='max_'+c,ascending=False)\
#     .index[0:100].values
#     top_bool=np.zeros(gene_cats_bpcc.shape[0],).astype(bool)
#     top_bool[index_of_max]=True    
    
#     hhgj
    n_top=sum(top_bool)
    print(n_top)
    
    table1=pd.DataFrame(index=comps2+['any comp','no comp'],columns=['Prevalence','p-value','odds ratio'])
    for co in comps2:
#         ratio_h=gene_cats_bpcc_highP[(gene_cats_bpcc_highP[co]!='')].shape[0]/\
#         gene_cats_bpcc_highP.shape[0]

#         ratio_l=gene_cats_bpcc_lowP[(gene_cats_bpcc_lowP[co]!='')].shape[0]/\
#         gene_cats_bpcc_lowP.shape[0]
# #         fdsfsd
#         print(c,'-',co,': ',ratio_h/ratio_l,gene_cats_bpcc_highP.shape[0],gene_cats_bpcc_lowP.shape[0])
        
        enr_ratio=gene_cats_bpcc2[top_bool & (gene_cats_bpcc2[co]!='')].shape[0]/\
        gene_cats_bpcc2[(gene_cats_bpcc2[co]!='')].shape[0]
        comp_bool=(gene_cats_bpcc2[co]!='').values
        oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, comp_bool))
    #     print(co, ':',gene_cats_bpcc[(gene_cats_bpcc['top58']==True) & (gene_cats_bpcc[co]!='')].shape[0],\
    #          ', ',gene_cats_bpcc[(gene_cats_bpcc[co]!='')].shape[0])
        
        table1.loc[co,['Prevalence','p-value','odds ratio']]=enr_ratio*100,pvalue,oddsratio
#         print(co,": ",np.round(enr_ratio*100,2),'%     ','pvalue:', np.round(pvalue,2),'  oddsratio:',np.round(oddsratio,2))
        table2.loc[co,c]=oddsratio
         
    
    enr_ratio=gene_cats_bpcc[top_bool & (gene_cats_bpcc['any_comps']!='')].shape[0]/\
    gene_cats_bpcc[(gene_cats_bpcc['any_comps']!='')].shape[0]
    any_oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, (gene_cats_bpcc['any_comps']!='').values))
#     print("any_comps: ",np.round(enr_ratio*100,2),'%     ','pvalue:', np.round(pvalue,2),'  oddsratio:',np.round(oddsratio,2))
    table1.loc['any comp',['Prevalence','p-value','odds ratio']]=enr_ratio*100,pvalue,oddsratio

    nocomp_enr_ratio=gene_cats_bpcc[(top_bool) & (gene_cats_bpcc['any_comps']=='')].shape[0]/\
gene_cats_bpcc[(gene_cats_bpcc['any_comps']=='')].shape[0]

    comps2=['mitochondria','cytoskeleton-actin','endoplasmic','RNA_nucleoli','DNA']
    comps2.remove(Chan_rev_dict[c])
    gene_cats_bpcc['rest_comps']=''
    for co in comps2:
        gene_cats_bpcc['rest_comps']=gene_cats_bpcc['rest_comps']+gene_cats_bpcc[co]    
    
    rest_enr_ratio=gene_cats_bpcc[(top_bool) & (gene_cats_bpcc['rest_comps']=='')].shape[0]/\
gene_cats_bpcc[(gene_cats_bpcc['rest_comps']=='')].shape[0]    
    rest_oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, (gene_cats_bpcc['rest_comps']!='').values))
    
    table.append(table1)
#     print('num top ('+ str(n_top)+')/total genes (912): ', np.round((n_top/912)*100,2),'%')
    table3.loc[c,['odds ratio','restComp_oddsratio','anyComp_oddsratio','top-ratio']]=\
    table1.loc[Chan_rev_dict[c],'odds ratio'],rest_oddsratio,\
    any_oddsratio,np.round((n_top/912)*100,2)
    
#     table3.loc[c,['Prevalence','anyComp-Prevalence','noComp-Prevalence','top-ratio']]=table1.loc[Chan_rev_dict[c],'Prevalence'],\
#     enr_ratio*100,nocomp_enr_ratio*100,np.round((n_top/912)*100,2)    
    
#     print(Chan_rev_dict[c],': ',table1.loc[Chan_rev_dict[c],['Prevalence']].values)
    
# # table3['dif']=table3['Prevalence']-table3['anyComp-Prevalence']
table3['dif']=table3['odds ratio']-table3['restComp_oddsratio']
# print(table3['dif'].min(),table3['dif'].sum())
table3

In [315]:
gene_cats_bpcc2=gene_cats_bpcc[(gene_cats_bpcc['top_channel']=='max_'+c) &(gene_cats_bpcc['max_'+c]>0.4)].reset_index(drop=True)

In [318]:
gene_cats_bpcc2[['max_'+cr for cr in c_r]].mean(axis=0)

max_AGP     0.507761
max_Mito    0.501436
max_DNA     0.493772
max_RNA     0.511070
dtype: float64

In [320]:
gene_cats_bpcc2[['max_'+c for c in Channelss]].median(axis=0)

max_DNA     0.475004
max_RNA     0.502518
max_AGP     0.491805
max_Mito    0.507971
max_ER      0.555582
dtype: float64